In [1]:
import cptac

In [ ]:
en = cptac.Endometrial()

In [8]:
col = cptac.Colon()

In [3]:
endo = en.get_genotype_all_vars("PIK3CA")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)


In [ ]:
#get the missense mutation hotspots
# abunance who have a mutation vs the wildtype (these are our two buckets)
# for every gene do a t test
# start with PIK3ca and go through every gene
#

In [9]:
colon = col.get_genotype_all_vars("PIK3CA")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 382)


In [13]:
missense_colcon = colon.loc[colon['Mutation'] == "nonsynonymous SNV"]
missense_colcon

Name,Mutation,Location,Mutation_Status
Patient_ID,,,
01CO022,nonsynonymous SNV,Q981R,Single_mutation
05CO003,nonsynonymous SNV,C420R,Multiple_mutation
05CO006,nonsynonymous SNV,E103G,Single_mutation
05CO028,nonsynonymous SNV,E545K,Single_mutation
05CO032,nonsynonymous SNV,H1047R,Single_mutation
05CO050,nonsynonymous SNV,E85G,Single_mutation
09CO005,nonsynonymous SNV,H1047R,Single_mutation
11CO018,nonsynonymous SNV,R108H,Single_mutation
11CO027,nonsynonymous SNV,E545K,Single_mutation


In [7]:
#pull out all the missense mutation
missense = endo.loc[endo['Mutation'] == "Missense_Mutation"]
# E542K
# E535K
# H1047R
missense

Name,Mutation,Location,Mutation_Status
Patient_ID,,,
C3L-00006,Missense_Mutation,p.E545K,Single_mutation
C3L-00032,Missense_Mutation,p.E545K,Single_mutation
C3L-00139,Missense_Mutation,p.Q546P,Multiple_mutation
C3L-00143,Missense_Mutation,p.R115L,Single_mutation
C3L-00156,Missense_Mutation,p.L628R,Multiple_mutation
C3L-00161,Missense_Mutation,p.R38C,Multiple_mutation
C3L-00362,Missense_Mutation,p.H1047R,Single_mutation
C3L-00563,Missense_Mutation,p.G118D,Multiple_mutation
C3L-00586,Missense_Mutation,p.F83S,Multiple_mutation
